#  WPS call Weather Regimes

The weather regimes process is devided into two parts. Based on a reference dataset (reanalyses or any kind of model dataset) the weatherregimes are calustered and appropriate statisiks calculated. Two processes can be used for the reference training (weatherregimes_reanalyse with a list of predifined reanalyses datasets or weatherregimes_model for any suitable dataset of your choise).
The output of this fist part is the forcing data for the second part. With the process weatherregimes_projection the trained statistic can be projected onto a dataset with appropriate variables (pressure data)

In [3]:
##############################
# load the required libraries 
#############################

from owslib.wps import WebProcessingService, monitorExecution, printInputOutput

In [4]:
#################################################
# connect to the compute provider hosting the WPS
#################################################

# wps_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps"
wps_url = "http://localhost:8093/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

In [5]:
##########################################
# print some information about the service
##########################################

print wps.identification.title
for process in wps.processes:
    print '%s : \t %s' % (process.identifier, process.abstract)

Flyingpigeon
subset_continents : 	 Returns only the selected polygon for each input dataset
subset_countries : 	 Returns only the selected polygon for each input dataset
subset_regionseurope : 	 Returns only the selected polygon for each input dataset
subset_points : 	 Extract Timeseries for specified coordinates from gridded datasets
indices_single : 	 This process calculates climate indices based on one single input variable.
weatherregimes_reanalyse : 	 Weather Regimes based on pressure patterns, fetching selected Realayses Datasets
weatherregimes_model : 	 Weather Regimes based on pressure patterns, fetching selected Realayses Datasets
weatherregimes_projection : 	 Weather Regimes detection based on trained reference statistics
analogs : 	 Search for days with analog pressure pattern
analogs_viewer : 	 Visualisation of text output of analogue process
segetalflora : 	 Species biodiversity of segetal flora. Imput files: variable:tas , domain: EUR-11 or EUR-44
sdm_gbifsearch : 	 Speci

In [6]:
#################################################
# print some information about a specific process
#################################################

# to recieve informations uncomment the follwing lines

#p = wps.describeprocess(identifier='weatherregimes_reanalyse')
#for input in p.dataInputs:
#  printInputOutput(input)
#  print '\n'

In [7]:
#####################
# execute the process
#####################

training = wps.execute(
    identifier="weatherregimes_reanalyse", 
    inputs=[ 
        ('season', 'DJF'),         ### uncomment the cell above 
        ('BBox','-80,22.5,50,70')  ### for more optional arguments
           ],
    async=True) 

In [8]:
##########################################################################
# the appropriate information are printed out here if the job is finished 
##########################################################################

monitorExecution(training, sleepSecs=5)
print training.getStatus()

for o in training.processOutputs:
    print o.reference

ProcessSucceeded
http://localhost:8090/wpsoutputs/flyingpigeon/output_pca-be00b5a4-4294-11e6-85d9-bf7ed4b432ac.dat
http://localhost:8090/wpsoutputs/flyingpigeon/Routput_graphic-be00b5a4-4294-11e6-85d9-bf7ed4b432ac.pdf
http://localhost:8090/wpsoutputs/flyingpigeon/output_classification-be00b5a4-4294-11e6-85d9-bf7ed4b432ac.Rdat
http://localhost:8090/wpsoutputs/flyingpigeon/output_netcdf-be00b5a4-4294-11e6-85d9-bf7ed4b432ac.nc


In [14]:
###############################################################
# Next step is to perform a projection with the trained output
###############################################################


### uncomment this lines for information:

# p = wps.describeprocess(identifier='weatherregimes_projection')
# for input in p.dataInputs:
#     printInputOutput(input)

In [15]:
#################################
# Define the places of your data
#################################
from os import listdir, getenv

HOME = getenv('HOME')

In [16]:
path = HOME+'/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/'

# can be a web url (http:// ...) as well
files = ['file://' + path + f for f in listdir(path)]
files

['file:///homel/nhempel/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_19900101-19991231.nc',
 'file:///homel/nhempel/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_19800101-19891231.nc',
 'file:///homel/nhempel/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_20000101-20051231.nc']

In [20]:
########################################
# and execute the process for projection
########################################

projection = wps.execute(
    identifier="weatherregimes_projection",
    inputs=[
       ("resource",files[0]),
       ("resource",files[1]),
       ("resource",files[2]),
       #("resource",files[3]),
       #("resource",files[4]),
       ("season",'DJF'),
       ("period", "19900101-20001231"),
       ("anualcycle", "19950101-19991231"),
       ("dat", training.processOutputs[0].reference),   #  this are the output urls of the reference training
       ("Rdat", training.processOutputs[2].reference),  #
       ("netCDF",training.processOutputs[3].reference)  # the reference data are used here for regridding and bbox
           ],
    async=True)

In [21]:
monitorExecution(projection, sleepSecs=5)
print projection.getStatus()

for o in projection.processOutputs:
    print o.reference

ProcessSucceeded
http://localhost:8090/wpsoutputs/flyingpigeon/output_pca-6074c620-4297-11e6-85d9-bf7ed4b432ac.dat
http://localhost:8090/wpsoutputs/flyingpigeon/Routput_graphic-6074c620-4297-11e6-85d9-bf7ed4b432ac.pdf
http://localhost:8090/wpsoutputs/flyingpigeon/output_classification-6074c620-4297-11e6-85d9-bf7ed4b432ac.Rdat
http://localhost:8090/wpsoutputs/flyingpigeon/output_netcdf-6074c620-4297-11e6-85d9-bf7ed4b432ac.nc


In [22]:
# Voila :-) 